In [1]:
pip install xgboost

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 MB 4.8 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [12]:
import boto3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from xgboost import XGBClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix, make_scorer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE, RFECV
from sklearn.svm import SVR

from cost_function import cost_function, cost_function_cutoff


s3= boto3.resource('s3')
bucket_name= 'daltondencklau-data445-bucket'
bucket= s3.Bucket(bucket_name)

file_key= 'turnover.csv'

bucket_object= bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

#reading the datefile
turnover = pd.read_csv(file_content_stream)
turnover.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


In [13]:
## creating dummy variables for sales and salary; removing the simutaneously
turnover= pd.concat([turnover.drop(columns=['sales', 'salary'], axis=1), pd.get_dummies(turnover[['sales', 'salary']])], axis=1)

#engineering features from the decisoon tree model (in_class_16)
turnover['interaction_1']=np.where(((turnover['satisfaction_level'] >= .115) &
                                   (turnover['satisfaction_level'] >= .465) &
                                   (turnover['number_project'] > 2.5)), 1, 0)

turnover['interaction_2']=np.where(((turnover['satisfaction_level'] >= .465) &
                                   (turnover['number_project'] <=2.5) &
                                   (turnover['last_evaluation'] <= .575)), 1, 0)
        
turnover['interaction_3']=np.where(((turnover['satisfaction_level'] >= .465) &
                                    (turnover['time_spend_company'] <=4.5) &
                                    (turnover['average_montly_hours'] <=290.5)), 1,0)

In [14]:
## Defining input and target variables
x = turnover[['satisfaction_level', 'number_project', 'time_spend_company', 'interaction_1', 'interaction_3']]
y = turnover['left']

## splitting the data
x_train, x_test, y_train, y_test= train_test_split(x, y, test_size = 0.2, stratify = y)

### RF GridsearchCV

In [15]:
## defining the hyperparameters
rf_param_grid = {'n_estimators': [100, 300, 500],
'min_samples_split': [10, 15],
'min_samples_leaf': [5, 7],
'max_depth' : [3, 5, 7]}

## defining the customized scoring function
my_score_function = make_scorer(cost_function, greater_is_better = True, need_proba = True)

## gridsearchCV model
rf_grid_search = GridSearchCV(estimator = RandomForestClassifier(), param_grid= rf_param_grid, cv = 3,
                              scoring = my_score_function, n_jobs = -1).fit(x_train, y_train)
    
## extracting best hyperparamter combinations
rf_md = rf_grid_search.best_estimator_

## prediciting on test
rf_test_pred = rf_md.predict_proba(x_test)[:,1]

## identify optimal cutoff
opt_cutoff = cost_function_cutoff(y_test, rf_test_pred)

## changing likelihoods to labels
rf_label = np.where(rf_test_pred < opt_cutoff , 0, 1)

## creating confusion matrix 
conf_matrix = confusion_matrix(y_test, rf_label)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 220, in __call__
    return self._score(
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 268, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
TypeError: cost_function() got an unexpected keyword argument 'need_proba'

  warnings.warn(
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:776: U

[[2217   69]
 [  81  633]]
Cost of RF model: 126000


In [16]:
## print results
print(conf_matrix)
print('Cost of RF model:', -1500*conf_matrix[1,0] - 1000*conf_matrix[0,1] + 500*conf_matrix[1,1])

[[2217   69]
 [  81  633]]
Cost of RF model: 126000


### XGB GridsearchCV

In [17]:
## defining the hyperparameters
xgb_param_grid = {'n_estimators': [500],
'max_depth': [3, 5, 7],
'min_child_weight': [5, 7],
'learning_rate': [0.01],
'gamma': [0.3, 0.1],
'subsample': [0.8, 1],
'colsample_bytree': [1]}

## defining customized scoring function
my_score_function = make_scorer(cost_function, greater_is_better = True, need_proba = True)

## gridsearchCV model
xgb_grid_search = GridSearchCV(estimator = XGBClassifier(), param_grid = xgb_param_grid, cv = 3,
                               scoring = my_score_function, n_jobs = -1).fit(x_train, y_train)
    
## extracting best hyperparamter combination
xgb_md = xgb_grid_search.best_estimator_

## prediciting on test
xgb_test_pred = xgb_md.predict_proba(x_test)[:,1]

## identify optimal cutoff
opt_cutoff = cost_function_cutoff(y_test, xgb_test_pred)

## changing likelihoodss to lables
xgb_label = np.where(xgb_test_pred < opt_cutoff , 0, 1)

## creating confusion matrix 
conf_matrix_2 = confusion_matrix(y_test, xgb_label)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/model_selection/_search.py:953: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan]
  warnings.warn(


In [18]:
## print results
print(conf_matrix_2)
print('Cost of RF model:', -1500*conf_matrix_2[1,0] - 1000*conf_matrix_2[0,1] + 500*conf_matrix_2[1,1])

[[2240   46]
 [  62  652]]
Cost of RF model: 187000


When predicting 'left', I would choose the XGBoost model because it gives the higher return value compared to the Random Forest model.